# Post-Midterm Report: Back to the Drawing Board 


Feature engineering and testing different model specifications. Models in this document are mostly of the non-NN variety to provide insight on how the metadata we intend to utilize in our models can improve our metrics. There are cases in which tf-idf weighting/good feature engineering is sufficient for specifying a model that outperforms the state of the art. This is something we can explore in our project.

In [1]:
import numpy as np 
import pandas as pd

from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble 

import xgboost, textblob, string 
from keras.preprocessing import text, sequence 
from keras import layers, models, optimizers

Using TensorFlow backend.
/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarn

In [2]:
# for reproducibility 
SEED = 1984
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True # for reproducible results 

NameError: name 'torch' is not defined

### Loading data, standardizing variables 

In [3]:
# load data from already split train/validation/test sets 
train_df = pd.read_csv('us_news_train.csv')
validation_df = pd.read_csv('us_news_validation.csv')
test_df = pd.read_csv('us_news_test.csv')

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (10,11,12,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# inspecting data 
train_df.head()
train_df.shape
train_df.dtypes
train_df['ideology'].value_counts()
train_df['outlet'].value_counts()

foxnews              13095
breitbart             9469
cnbc                  7029
huffingtonpost        6711
infowars              5244
dailykos              4778
bbcnews               4726
washingtonpost        4653
thehill               4237
wallstreetjournal     2867
abcnews               2666
yahoous               2440
thinkprogress         2439
nytimes               2321
motherjones           2114
newsweek              1878
guardian              1419
townhall              1257
politico               657
Name: outlet, dtype: int64

In [5]:
# changing variable names for validation set 
validation_df.columns = validation_df.columns.str.replace(r"validate.","")
validation_df.dtypes

outlet           object
outlet_url       object
datetime         object
url_orig         object
headline         object
description      object
author           object
domain           object
topic_tags       object
text             object
section          object
news_keywords    object
subsection       object
paywall          object
provider         object
ideology         object
dtype: object

### Label encoding and setting target variable `['ideology']`

In [6]:
# label encoding  
le = LabelEncoder() # creating labelencoder object 
# converting categorical target variable 'ideology' into numeric 
train_df['ideology'] = le.fit_transform(train_df['ideology'])
validation_df['ideology'] = le.fit_transform(validation_df['ideology'])
test_df['ideology'] = le.fit_transform(test_df['ideology'])
# converting other categorical columns into numeric 
train_df['outlet'] = le.fit_transform(train_df['outlet'])
validation_df['outlet'] = le.fit_transform(validation_df['outlet'])
test_df['outlet'] = le.fit_transform(test_df['outlet'])
# set target variable: ideology 
target = train_df['ideology']
target = validation_df['ideology']
target = test_df['ideology']

### Feature Engineering 

In [24]:
validation_df.dtypes
train_df.dtypes

outlet            int64
outlet_url       object
datetime         object
url_orig         object
headline         object
description      object
author           object
domain           object
topic_tags       object
text             object
section          object
news_keywords    object
subsection       object
paywall          object
provider         object
ideology          int64
dtype: object

In [7]:
# converting dtype objects to unicode strings
# train_df['headline','text','topic_tags','description'] = train_df['headline','text','topic_tags','description'].astype('unicode')
# validation_df['headline','text','topic_tags','description'] = validation_df['headline','text','topic_tags','description'].astype('unicode')

KeyError: ('headline', 'text', 'topic_tags', 'description')

In [43]:
# cleaning text here?? 

#### Count Vectors as Features

In [33]:
# creating count vectorizer objects
# count_vect = CountVectorizer(analyzer='word',token_pattern=r'w{1,}',max_features=5000)
# count_vect.fit(train_df['headline'])
#count_vect.fit(validation_df['headline','text','topic_tags','description'])
# transforming training and validation data using count vectorizer object 
# xtrain_count = count_vect.transform(train_df)
#xtrain_count = count_vect.transform(validation_df)
v = TfidfVectorizer(decode_error='replace', encoding='utf-8')
x = v.fit_transform(train_df['headline'].values.astype(str))
print(x.toarray())

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [31]:
# word-level tfidf 
tfidf_vect=TfidfVectorizer(analyzer='word',token_pattern=r'\w{1,}',max_features=5000)
tfidf_vect.fit(train_df['headline'])

ValueError: np.nan is an invalid document, expected byte or unicode string.